In [1]:
!pip install gradio

In [3]:
# ==========================================
# Heart Disease Prediction - Gradio UI
# ==========================================

import gradio as gr
import numpy as np
import pandas as pd
import pickle
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer

# --------------------------------------------------
# Load models, transformer, and scaler
# --------------------------------------------------
# Make sure these are saved from your training notebook:
# pickle.dump(knn_model, open("knn_model.pkl", "wb"))
# pickle.dump(dt_model, open("decision_tree_model.pkl", "wb"))
# pickle.dump(transformer, open("transformer.pkl", "wb"))
# pickle.dump(scaler, open("scaler.pkl", "wb"))

with open("knn_model.pkl", "rb") as f:
    knn_model = pickle.load(f)

with open("dt_model.pkl", "rb") as f:
    dt_model = pickle.load(f)

with open("transformer.pkl", "rb") as f:
    transformer = pickle.load(f)

with open("scaler.pkl", "rb") as f:
    scaler = pickle.load(f)

# --------------------------------------------------
# Define the prediction function
# --------------------------------------------------

def predict_heart_disease(
    BMI, Smoking, AlcoholDrinking, Stroke, PhysicalHealth,
    MentalHealth, DiffWalking, Sex, AgeCategory, Race,
    Diabetic, PhysicalActivity, GenHealth, SleepTime,
    Asthma, KidneyDisease, SkinCancer, model_choice
):
    # Build dataframe from inputs
    input_dict = {
        "BMI": [float(BMI)],
        "Smoking": [int(Smoking)],
        "AlcoholDrinking": [int(AlcoholDrinking)],
        "Stroke": [int(Stroke)],
        "PhysicalHealth": [float(PhysicalHealth)],
        "MentalHealth": [float(MentalHealth)],
        "DiffWalking": [int(DiffWalking)],
        "Sex": [int(Sex)],
        "AgeCategory": [AgeCategory],
        "Race": [Race],
        "Diabetic": [int(Diabetic)],
        "PhysicalActivity": [int(PhysicalActivity)],
        "GenHealth": [GenHealth],
        "SleepTime": [float(SleepTime)],
        "Asthma": [int(Asthma)],
        "KidneyDisease": [int(KidneyDisease)],
        "SkinCancer": [int(SkinCancer)]
    }

    input_df = pd.DataFrame(input_dict)

    # Apply same preprocessing as during training
    transformed = transformer.transform(input_df)
    scaled = scaler.transform(transformed)

    # Choose model
    model = knn_model if model_choice == "K-Nearest Neighbors" else dt_model

    # Predict
    pred = model.predict(scaled)[0]
    result = "💔 OO NO!!!   Likely to have heart disease" if pred == 1 else "💖 YEEAHHH!  Not likely to have heart disease"
    return result


# --------------------------------------------------
# Build Gradio Interface with feature explanations
# --------------------------------------------------

interface = gr.Interface(
    fn=predict_heart_disease,
    inputs=[
        gr.Number(
            label="BMI (Body Mass Index)",
            info="Body mass index (18–50 recommended range). Higher values indicate overweight or obesity."
        ),
        gr.Radio(
            [0, 1],
            label="Smoking",
            info="1 = Has smoked at least 100 cigarettes in life, 0 = Has not smoked."
        ),
        gr.Radio(
            [0, 1],
            label="Alcohol Drinking",
            info="1 = Drinks alcohol heavily, 0 = Does not."
        ),
        gr.Radio(
            [0, 1],
            label="Stroke",
            info="1 = Has experienced a stroke, 0 = Has not."
        ),
        gr.Number(
            label="Physical Health (days)",
            info="Number of days in past 30 days physical health was not good (0–30)."
        ),
        gr.Number(
            label="Mental Health (days)",
            info="Number of days in past 30 days mental health was not good (0–30)."
        ),
        gr.Radio(
            [0, 1],
            label="Difficulty Walking",
            info="1 = Has serious difficulty walking or climbing stairs, 0 = No difficulty."
        ),
        gr.Radio(
            [0, 1],
            label="Sex",
            info="1 = Male, 0 = Female."
        ),
        gr.Dropdown(
            ["18-24", "25-29", "30-34", "35-39", "40-44",
             "45-49", "50-54", "55-59", "60-64", "65-69",
             "70-74", "75-79", "80+"],
            label="Age Category",
            info="Age group of the respondent."
        ),
        gr.Dropdown(
            ["White", "Black", "Asian", "American Indian/Alaskan Native", "Hispanic", "Other"],
            label="Race",
            info="Race or ethnicity of the individual."
        ),
        gr.Radio(
            [0, 1],
            label="Diabetic",
            info="1 = Has diabetes or told by doctor to have it, 0 = Does not."
        ),
        gr.Radio(
            [0, 1],
            label="Physical Activity",
            info="1 = Has performed physical activity in past 30 days, 0 = Has not."
        ),
        gr.Dropdown(
            ["Poor", "Fair", "Good", "Very good", "Excellent"],
            label="General Health",
            info="Overall self-rated health."
        ),
        gr.Number(
            label="Sleep Time (hours per day)",
            info="Average hours of sleep per day (typically 4–12)."
        ),
        gr.Radio(
            [0, 1],
            label="Asthma",
            info="1 = Has asthma, 0 = Does not."
        ),
        gr.Radio(
            [0, 1],
            label="Kidney Disease",
            info="1 = Has kidney disease (excluding stones or bladder infection), 0 = Does not."
        ),
        gr.Radio(
            [0, 1],
            label="Skin Cancer",
            info="1 = Has skin cancer, 0 = Does not."
        ),
        gr.Radio(
            ["K-Nearest Neighbors", "Decision Tree"],
            label="Select Prediction Model",
            info="Choose which trained model to use for prediction.",
            value="K-Nearest Neighbors"
        )
    ],
    outputs=gr.Textbox(label="Prediction"),
    title="❤️ Heart Disease Prediction App",
    description=(
        "Enter patient information to estimate the likelihood of heart disease. "
        "This tool uses machine learning models trained on CDC Heart Disease data. "
        "All inputs should be realistic, e.g., BMI 18–50, Sleep 4–12 hours, Health days 0–30."
    )
)

interface.launch()


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
